In [3]:
''' 
Feature_Engineer:
    1. 提取 train.txt 中所有的 Nodes 和 Edges 关系
    2. 利用上述关系生成 pos_data 和 neg_data （已经生成，直接读取)
    3. 利用（1）中生成的 Nodes 和 Edges 生成图

'''

' \nFeature_Engineer:\n    1. 提取 train.txt 中所有的 Nodes 和 Edges 关系\n    2. 利用上述关系生成 pos_data 和 neg_data （已经生成，直接读取)\n    3. 利用（1）中生成的 Nodes 和 Edges 生成图\n\n'

In [4]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle
import math

dirname = 'data/'
def save_obj(obj, name ):
    with open(dirname+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( dirname + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
with open("data/train.txt", "r") as f:
     train_data = f.readlines()
Nodes = []
Edges = []
for i in range(len(train_data)):
    #if i%100 == 0:
        #print(i)
    nodes_list = [int(n) for n in train_data[i].split()]
    for node in nodes_list:
        Nodes.append(node)
    for node in nodes_list[1:]:
        Edges.append((nodes_list[0],node))
Nodes = set(Nodes)
# 生成Nodes和Edges，直接从train.txt 读取就可以

In [6]:
# 读取pos_data 和 neg_data，已经在本地保存，可以直接读取

pos_data = load_obj("pos_data")
neg_data = load_obj("neg_data")

In [7]:
# train.txt 文件已经读取， pos_data 和 neg_data 也已经读取
print('Nodes number:\t','Edges number:')
print(len(Nodes), '\t',len(Edges), '\n')
print('posdata_length: ', 'negdata_length:')
print(len(pos_data),'\t', len(neg_data), '\n')

print('first 10 pieces of pos_data and neg_data:')
print(pos_data[:10],'\n')
print(neg_data[:10])



Nodes number:	 Edges number:
4867136 	 24004361 

posdata_length:  negdata_length:
24004361 	 23999986 

first 10 pieces of pos_data and neg_data:
[[(540762, 1912140), 1], [(540762, 1537559), 1], [(540762, 3091331), 1], [(540762, 2757277), 1], [(540762, 3237295), 1], [(540762, 1070876), 1], [(540762, 4008078), 1], [(540762, 1824878), 1], [(540762, 1005927), 1], [(540762, 2703564), 1]] 

[[(3480985, 2887932), 0], [(3480985, 888056), 0], [(3480985, 1084524), 0], [(3480985, 2144717), 0], [(3480985, 353911), 0], [(3480985, 1329443), 0], [(3480985, 288458), 0], [(3480985, 2630047), 0], [(3480985, 3966258), 0], [(3480985, 453298), 0]]


In [8]:
G = nx.Graph()
G.add_nodes_from(Nodes)
G.add_edges_from(Edges)
# Edges and Nodes are all from train.txt (pos_data)
print('Graph has been built!')

Graph has been built!


In [9]:
# 保存G对象到本地
save_obj(G, 'graph')
print('graph has been saved')

graph has been saved


In [10]:
# build hash set for Edges , hash set is more quick for calculating
hash_edges = set(Edges)

In [11]:
#print(Edges[:10])
# let's try a test node:
test_node = 3043
test_neig = sorted(nx.all_neighbors(G, test_node))
print(len(test_neig))

num_in = 0
num_out = 0
for one_neig in test_neig:
    if (3043, one_neig) in hash_edges:
        num_out += 1
    if (one_neig, 3043) in hash_edges:
        num_in +=1
print(num_in, num_out)
    

218
209 55


In [ ]:
'''
debug: 为什么所有的边都被保存为了inbound？

solution:  代码从trian.txt中读取了Edges，错在“直接把Edges转化为hash set 然后判断(e, node)是否存在”

'''

'''
生成pre_features: pre_features 需要包含哪些特征？

1. neighbors number 
2. log(nei_number)
3. all_neighbors
4. inbound
5. inbound_num
6. outbound
7. outboud_num
    
'''
pre_features = {}
for i,node in enumerate(list(Nodes)):
    #if i % 50000 == 0:
        #print('progress:',100*i/4867136, '%')    # uncomment if u want progress info
    num_neig = len(sorted(nx.all_neighbors(G, node)))                 # 1
    log_neig = (1. / math.log(num_neig+1)) if num_neig != 0 else 0    # 2
    neig = sorted(nx.all_neighbors(G, node))                          # 3
    
    inbound = []
    outbound = []
    for e in list(neig):
        if (e,node) in hash_edges:
            inbound.append(e)
        if (node,e) in hash_edges:
            outbound.append(e)
    
    pre_features[node] = [num_neig, log_neig, neig, inbound, len(inbound), outbound, len(outbound)]

In [25]:
save_obj(pre_features, 'pre_features_v2')

In [ ]:
'''
Next we combine the pos_data and neg_data to one file:
SBdata
'''

In [ ]:
import pickle
import numpy
import pandas

obj_data_dir = "../../data/"
def save_obj(obj, name ):
    with open(obj_data_dir + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(obj_data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)
# load data
#training_edges=load_obj('SBdata')

if pos_data is None:
    pos_data = load_obj("pos_data")
if neg_data is None:
    neg_data = load_obj("neg_data")

pos = pos_data
neg = neg_data

print(len(pos))
print(neg[1])

filename = "SBdata"
SBdata = []
for element in pos:
    SBdata.append(element)
for element in neg:
    SBdata.append(element)

In [ ]:
save_obj(SBdata, 'SBdata')